<a href="https://colab.research.google.com/github/KimwWoYoung/kimwooyoung/blob/main/%E3%85%83%E3%85%83%E3%85%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install gdown
!gdown "https://drive.google.com/uc?id=1me0Hb5y2YZwEqCTiQvObR6_22S6hL9B6"
!unzip "archive (3).zip"

In [ ]:
pip install keras --upgrade

In [ ]:
from keras.layers import BatchNormalization
from tensorflow import keras
from tensorflow.keras import layers
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from keras.layers import Dropout
#from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.models import Sequential
import numpy as np

In [ ]:
# 이미지 데이터를 불러올 디렉토리 경로를 설정합니다.
train_dir = "cifar10/train"
test_dir = "cifar10/test"

In [ ]:
# Step 1: Load the data using ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir ,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical')

In [ ]:
# Step 2: Preprocess the data
x_train = np.concatenate([x for x, y in train_generator], axis=0).astype('float32') / 255
y_train = np.concatenate([y for x, y in train_generator], axis=0)

x_test= np.concatenate([x for x, y in test_generator], axis=0).astype('float32') / 255
y_test = np.concatenate([y for x, y in test_generator], axis=0)

In [ ]:
x_train.shape

In [ ]:
# Step 3: Load the neural network model
model = keras.Sequential([
    Dense(512, activation='relu', input_shape=(32, 32, 3)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Flatten(),
    Dense(10, activation='softmax')
])

In [ ]:
# Step 4: Compile the model
model.compile(optimizer=keras.optimizers.RMSprop(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Step 5: Train the model with validation_split=0.1
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=256,
                    validation_split=0.1)

In [ ]:
for epoch in [10, 30, 50]:
    for batch_size in [16, 32, 64]:
        model.compile(optimizer=keras.optimizers.Adam(1e-3),
                      loss="binary_crossentropy",
                      metrics=["accuracy"])
        history = model.fit(x_train, y_train,
                            epochs=epoch,
                            batch_size=batch_size,
                            validation_split=0.1,
                            verbose=0)
        print(f"Epoch: {epoch}, Batch size: {batch_size} - val_accuracy: {history.history['val_accuracy'][-1]}")

In [ ]:
# Step 6: Plot the loss and val_loss graphs
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Step 7: Plot the accuracy and val_accuracy graphs
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Step 8: Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)